In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F

transform = transforms.Compose([
                transforms.ToTensor()
            ])

train = datasets.CIFAR10('./data', train=True, download=True, transform=transform)
test = datasets.CIFAR10('./data', train=False, download=True, transform=transform)

trainset = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=128, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3*32*32, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        
        return F.log_softmax(x, dim=1)

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=3072, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)


In [16]:
import torch.optim as optim

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [23]:
for epoch in range(10):
    for data in trainset:
        X, y = data
        net.zero_grad()
        output = net(X.reshape(-1, 3072))
        loss = loss_function(output, y)
        loss.backward()
        optimizer.step()
    print(loss)
        

tensor(1.3213, grad_fn=<NllLossBackward0>)
tensor(1.3622, grad_fn=<NllLossBackward0>)
tensor(1.2613, grad_fn=<NllLossBackward0>)
tensor(1.3663, grad_fn=<NllLossBackward0>)
tensor(1.5061, grad_fn=<NllLossBackward0>)
tensor(1.4303, grad_fn=<NllLossBackward0>)
tensor(1.1888, grad_fn=<NllLossBackward0>)
tensor(1.5257, grad_fn=<NllLossBackward0>)
tensor(1.2993, grad_fn=<NllLossBackward0>)
tensor(1.3874, grad_fn=<NllLossBackward0>)


In [22]:
correct = 0
total = 0

with torch.no_grad():
    for data in testset:
        X, y = data
        output = net(X.reshape(-1, 3072))
        
        _, predicted = torch.max(output, 1)
        correct += (predicted == y).sum().item()
        total += y.size(0)

print(f"Accuracy: {correct/total:.3f}")

Accuracy: 0.492
